In [1]:
!pip install ipywidgets >> /dev/null
!pip install pyaudio >> /dev/null

In [2]:
import json
import pyaudio
import subprocess
from queue import Queue
from threading import Thread
import ipywidgets as widgets
from vosk import Model, KaldiRecognizer

In [3]:
messages = Queue()
recordings = Queue()

In [ ]:
from IPython.display import display
record_button = widgets.Button(
    description="Record",
    disavbled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disavbled=False,
    button_style="warning",
    icon="stop"
)


output = widgets.Output()

def start_recording(data):
    messages.put(True)

    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()

        transcribe  = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)
display(record_button, stop_button, output)

NameError: name 'widgets' is not defined

In [15]:
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'HDA Intel PCH: ALC257 Analog (hw:0,0)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.005804988662131519, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': 0.034829931972789115, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'HDA Intel PCH: HDMI 0 (hw:0,3)', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 8, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'HDA Intel PCH: HDMI 1 (hw:0,7)', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 8, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.034829931972789115, '

In [23]:
CHANNELS = 2
FRAME_RATE = 16000
RECORD_SECONDs = 1
AUDIO_FORMAT = pyaudio.paInt16

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=9,
                    frames_per_buffer=chunk)
    
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE * RECORD_SECONDs) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [7]:
model = Model(model_name="vosk-model-uk-v3")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/nickolay/.cache/vosk/vosk-model-uk-v3/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /home/nickolay/.cache/vosk/vosk-model-uk-v3/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from /home/nickolay/.cache/vosk/vosk-model-uk-v3/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/nickolay/.cache/vosk/vosk-model-uk-v3/graph/phones/word_

In [8]:
def map_text_to_number(text):
    # Mapping Ukrainian number words to digits
    number_dict = {
        'один': 1,
        'два': 2,
        'три': 3,
        'чотири': 4,
        'п’ять': 5,
        'шість': 6,
        'сім': 7,
        'вісім': 8,
        'дев’ять': 9
    }
    
    # Normalize the text to remove any whitespace or punctuation
    normalized_text = text.strip().lower()
    
    # Return the corresponding number or a message if the word is not found
    return number_dict.get(normalized_text, 0)

# Example usage



In [25]:
def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()

        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        print(result)
        text = json.loads(result)["text"]
        
        output.append_stdout(text)